## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = app_df.drop(['EIN','SPECIAL_CONSIDERATIONS','STATUS'],1)

<ipython-input-3-63ee045b5c4f>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  app_df = app_df.drop(['EIN','SPECIAL_CONSIDERATIONS','STATUS'],1)


In [4]:
# Determine the number of unique values in each column.
app_df.nunique()

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
name_counts = app_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
name_types_to_replace = list(name_counts[name_counts < 10].index)
# Replace in dataframe
for app in name_types_to_replace:
    app_df['NAME'] = app_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
app_df['NAME'].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = app_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1000]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_counts[classification_counts < 4000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
df_app_dummies = pd.get_dummies(app_df,dtype=float)

In [11]:
# Split our preprocessed data into our features and target arrays
y = df_app_dummies['IS_SUCCESSFUL'].values
X = df_app_dummies.drop(['IS_SUCCESSFUL'],1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

<ipython-input-11-fd02745fe80e>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df_app_dummies.drop(['IS_SUCCESSFUL'],1).values


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  66
hidden_nodes_layer2 =  9
hidden_nodes_layer3 =  3
hidden_nodes_layer4 =  9

nn = tf.keras.models.Sequential()

# First hidden layer
# Changing activation method from relu to tanh, selu, para_relu
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
# Changing activation method from relu to 
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="selu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 66)                17820     
                                                                 
 dense_1 (Dense)             (None, 9)                 603       
                                                                 
 dense_2 (Dense)             (None, 3)                 30        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 18,457
Trainable params: 18,457
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
# Increasing number of epochs from 100 to 103
fit_model = nn.fit(X_train_scaled,y_train,epochs=103)

Epoch 1/103
804/804 [==============================] - 3s 2ms/step - loss: 0.5023 - accuracy: 0.7544
Epoch 2/103
804/804 [==============================] - 2s 2ms/step - loss: 0.4582 - accuracy: 0.7797
Epoch 3/103
804/804 [==============================] - 2s 3ms/step - loss: 0.4492 - accuracy: 0.7809
Epoch 4/103
804/804 [==============================] - 3s 3ms/step - loss: 0.4468 - accuracy: 0.7810
Epoch 5/103
804/804 [==============================] - 2s 3ms/step - loss: 0.4452 - accuracy: 0.7818
Epoch 6/103
804/804 [==============================] - 2s 2ms/step - loss: 0.4446 - accuracy: 0.7818
Epoch 7/103
804/804 [==============================] - 2s 2ms/step - loss: 0.4434 - accuracy: 0.7835
Epoch 8/103
804/804 [==============================] - 2s 2ms/step - loss: 0.4421 - accuracy: 0.7844
Epoch 9/103
804/804 [==============================] - 2s 2ms/step - loss: 0.4412 - accuracy: 0.7845
Epoch 10/103
804/804 [==============================] - 2s 2ms/step - loss: 0.4378 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4553 - accuracy: 0.7876 - 1s/epoch - 5ms/step
Loss: 0.4553055167198181, Accuracy: 0.7876384854316711


In [18]:
# Export our model to HDF5 file
nn.save('alphabet_soup_charity_optimized.h5')